In [1]:
!pip install vaderSentiment


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.8 MB/s eta 0:00:00


In [2]:
# 📘 generate_data.ipynb
# ======================
# Script to load, clean, enrich, and save EdTech recommendation datasets

import pandas as pd
import os
import re
import string
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [3]:
# ---------------------------------------------
# 📁 Data Loading
# ---------------------------------------------
def load_csv(filepath: str):
    if not os.path.exists(filepath):
        print(f"❌ File not found: {filepath}")
        return None
    try:
        df = pd.read_csv(filepath)
        print(f"✅ Loaded {filepath} - shape: {df.shape}")
        return df
    except Exception as e:
        print(f"⚠️ Error loading {filepath}: {e}")
        return None

# ---------------------------------------------
# 🔧 Validation and Cleaning
# ---------------------------------------------
def validate_columns(df: pd.DataFrame, required_cols: list, name: str = "Data") -> bool:
    missing = [col for col in required_cols if col not in df.columns]
    if missing:
        print(f"❌ Missing columns in {name}: {missing}")
        return False
    return True

def clean_text(text: str) -> str:
    if pd.isnull(text):
        return ""
    text = text.lower()
    text = re.sub(rf"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub(r'\s+',' ', text).strip()
    return text

def preprocess_feedback_df(df: pd.DataFrame) -> pd.DataFrame:
    # Rename columns to expected names
    df = df.rename(columns={
        'CustomerID': 'learner_id',
        'Country': 'trainer_id',  # Assuming each country maps to a school
        'SatisfactionScore': 'rating'
    })

    # Add placeholder feedback text if missing
    if 'feedback_text' not in df.columns:
        df['feedback_text'] = "Customer rated satisfaction as " + df['rating'].astype(str)

    # Clean and enrich
    required_cols = ['learner_id', 'trainer_id', 'rating', 'feedback_text']
    if not validate_columns(df, required_cols, name="Feedback Data"):
        return df

    df = df.dropna(subset=['learner_id', 'trainer_id', 'rating'])
    df['feedback_text'] = df['feedback_text'].fillna("")
    df['cleaned_feedback'] = df['feedback_text'].apply(clean_text)
    df = df.drop_duplicates(subset=['learner_id', 'trainer_id', 'feedback_text'])

    print(f"✅ Feedback data cleaned. Shape: {df.shape}")
    return df

def preprocess_trainer_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns={
        'DBN': 'trainer_id',
        'School Name': 'name'
    })

    if 'domain' not in df.columns:
        df['domain'] = 'Unknown'

    required_cols = ['trainer_id', 'name', 'domain']
    if not validate_columns(df, required_cols, name="Trainer Data"):
        return df

    df = df.dropna(subset=['trainer_id', 'name'])
    df = df.drop_duplicates(subset=['trainer_id'])

    print(f"✅ Trainer data cleaned. Shape: {df.shape}")
    return df



In [4]:
# ---------------------------------------------
# 📊 Sentiment Analysis
# ---------------------------------------------
def get_textblob_polarity(text: str) -> float:
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return 0.0

def get_vader_compound(text: str) -> float:
    analyzer = SentimentIntensityAnalyzer()
    try:
        return analyzer.polarity_scores(text)['compound']
    except:
        return 0.0

def label_sentiment(score: float, pos_thres=0.1, neg_thres=-0.1) -> str:
    if score > pos_thres:
        return "Positive"
    elif score < neg_thres:
        return "Negative"
    else:
        return "Neutral"

def add_sentiment_columns(df: pd.DataFrame, text_column='cleaned_feedback') -> pd.DataFrame:
    df['tb_score'] = df[text_column].apply(get_textblob_polarity)
    df['tb_sentiment'] = df['tb_score'].apply(label_sentiment)

    df['vader_score'] = df[text_column].apply(get_vader_compound)
    df['vader_sentiment'] = df['vader_score'].apply(label_sentiment)

    print(f"✅ Added sentiment scores using TextBlob and VADER. Shape: {df.shape}")
    return df

def compute_avg_sentiment_per_trainer(df: pd.DataFrame, score_col='vader_score') -> pd.DataFrame:
    return df.groupby('trainer_id')[score_col].mean().reset_index().rename(columns={score_col: 'avg_sentiment'})

# ---------------------------------------------
# 💾 Save Output
# ---------------------------------------------
def save_processed(df: pd.DataFrame, path: str) -> None:
    try:
        df.to_csv(path, index=False)
        print(f"💾 Saved cleaned data to {path}")
    except Exception as e:
        print(f"❌ Failed to save {path}: {e}")


In [5]:
# -----------------------------
# 🚀 Run the ETL Pipeline
# -----------------------------

# Ensure output folder exists
os.makedirs("data", exist_ok=True)

# --- Step 1: Load raw CSV files ---
trainer_raw = load_csv("/kaggle/input/feedback-dataset/2019_Public_Data_File_-_Students.csv")
feedback_raw = load_csv("/kaggle/input/customer-feedback-and-satisfaction/customer_feedback_satisfaction.csv")


✅ Loaded /kaggle/input/feedback-dataset/2019_Public_Data_File_-_Students.csv - shape: (1829, 11)
✅ Loaded /kaggle/input/customer-feedback-and-satisfaction/customer_feedback_satisfaction.csv - shape: (38444, 11)


In [6]:
feedback_raw.head()

,CustomerID,Age,Gender,Country,Income,ProductQuality,ServiceQuality,PurchaseFrequency,FeedbackScore,LoyaltyLevel,SatisfactionScore
0,1,56,Male,UK,83094,5,8,5,Low,Bronze,100.0
1,2,69,Male,UK,86860,10,2,8,Medium,Gold,100.0
2,3,46,Female,USA,60173,8,10,18,Medium,Silver,100.0
3,4,32,Female,UK,73884,7,10,16,Low,Gold,100.0
4,5,60,Male,UK,97546,6,4,13,Low,Bronze,82.0


In [7]:
trainer_raw.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,DBN,School Name,Total Parent \nResponse Rate %,Total Teacher Response Rate %,Total Student Response Rate %,Collaborative Teachers Score,Effective School Leadership Score,Rigorous Instruction Score,Supportive Environment Score,Strong Family-Community Ties Score,Trust Score
0,01M015,P.S. 015 ROBERTO CLEMENTE,91,100,NaN,4.10,4.19,4.02,NaN,4.18,3.96
1,01M019,P.S. 019 ASHER LEVY,100,93,NaN,4.53,4.51,4.80,NaN,4.66,3.76
2,01M020,P.S. 020 ANNA SILVER,58,90,NaN,2.71,2.98,1.92,NaN,3.84,3.14
3,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,29,100,95.0,2.69,2.59,2.14,NaN,3.67,2.38
4,01M063,THE STAR ACADEMY - P.S.63,80,100,NaN,NaN,NaN,NaN,NaN,NaN,3.77


In [8]:
# --- Step 2: Clean feedback and trainer data ---
feedback_clean = preprocess_feedback_df(feedback_raw)
trainer_clean = preprocess_trainer_df(trainer_raw)


✅ Feedback data cleaned. Shape: (38444, 13)
✅ Trainer data cleaned. Shape: (1829, 12)


In [9]:
# --- Step 3: Enrich feedback with sentiment scores ---
feedback_sentiment = add_sentiment_columns(feedback_clean)

# --- Step 4: Aggregate average sentiment per trainer ---
avg_sentiment_df = compute_avg_sentiment_per_trainer(feedback_sentiment)


✅ Added sentiment scores using TextBlob and VADER. Shape: (38444, 17)


In [10]:

# --- Step 5: Save cleaned datasets ---
save_processed(feedback_sentiment, "/kaggle/working/preprocessed_feedback.csv")
save_processed(trainer_clean, "/kaggle/working/preprocessed_trainers.csv")
save_processed(avg_sentiment_df, "/kaggle/working/avg_sentiment_per_trainer.csv")

print("✅ All processed data files saved in /kaggle/working/")


💾 Saved cleaned data to /kaggle/working/preprocessed_feedback.csv
💾 Saved cleaned data to /kaggle/working/preprocessed_trainers.csv
💾 Saved cleaned data to /kaggle/working/avg_sentiment_per_trainer.csv
✅ All processed data files saved in /kaggle/working/
